## 以 mnist 手写数字识别为例
为了简单方便，我们以 单个神经元为例进行

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import math
import matplotlib.pyplot as plt
import os


mnist = input_data.read_data_sets('./mnist_dataset/', one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist_dataset/t10k-images-idx3-ubyte.gz
Extracting ./mnist_dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


### 模型建立与训练

In [2]:
x = tf.placeholder(tf.float32, [None, 784], name='X')
y = tf.placeholder(tf.float32, [None, 10], name='Y')
image_shape_input = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image('input', image_shape_input, 9)

W = tf.Variable(tf.random_normal([784, 10]), name='W')
b = tf.Variable(tf.zeros([10]), name='b')

forward = tf.matmul(x, W) + b
pred = tf.nn.softmax(forward)

epochs = 100
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
display_step = 1
learning_rate = 0.01
save_step = 5

ckpt_dir = './mnist_Tensorboard_tutorial_ckpt'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

loss_function = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.histogram('forward', forward)
tf.summary.scalar('loss', loss_function)
tf.summary.scalar('accuracy', accuracy)

saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
merged_summary_op = tf.summary.merge_all()
writer = tf.summary.FileWriter('./Tensorboard_tutorial_log', sess.graph)
for epoch in range(epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        summary_str = sess.run(merged_summary_op, feed_dict={x:xs, y:ys})
        writer.add_summary(summary_str, epoch)
        sess.run(optimizer, feed_dict={x:xs, y:ys})
    val_loss, val_acc = sess.run([loss_function, accuracy],
                         feed_dict={x:mnist.validation.images, y:mnist.validation.labels})
    if (epoch + 1) % display_step == 0:
        print('Epoch:{:03d}'.format(epoch + 1), 
              'val_loss=%6f' % val_loss, 
              'val_accuracy=%4f' % val_acc,
              'learning_rate=%.4f' % learning_rate)
    if (epoch + 1) % save_step == 0 or (epoch + 1) == epochs:
        saver.save(sess, os.path.join(ckpt_dir, 'mnist_model_{0:03d}.ckpt'.format(epoch + 1)))
        print('INFO:mnist_model_{0:03d}.ckpt saved'.format(epoch + 1))
print('INFO:Train Finished!')

Epoch:001 val_loss=6.032761 val_accuracy=0.251600 learning_rate=0.0100
Epoch:002 val_loss=3.616500 val_accuracy=0.431600 learning_rate=0.0100
Epoch:003 val_loss=2.586402 val_accuracy=0.540200 learning_rate=0.0100
Epoch:004 val_loss=2.075874 val_accuracy=0.613400 learning_rate=0.0100
Epoch:005 val_loss=1.780467 val_accuracy=0.658400 learning_rate=0.0100
INFO:mnist_model_005.ckpt saved
Epoch:006 val_loss=1.590022 val_accuracy=0.690600 learning_rate=0.0100
Epoch:007 val_loss=1.453197 val_accuracy=0.712200 learning_rate=0.0100
Epoch:008 val_loss=1.350329 val_accuracy=0.729800 learning_rate=0.0100
Epoch:009 val_loss=1.270867 val_accuracy=0.743600 learning_rate=0.0100
Epoch:010 val_loss=1.205626 val_accuracy=0.756400 learning_rate=0.0100
INFO:mnist_model_010.ckpt saved
Epoch:011 val_loss=1.151576 val_accuracy=0.768200 learning_rate=0.0100
Epoch:012 val_loss=1.104758 val_accuracy=0.777400 learning_rate=0.0100
Epoch:013 val_loss=1.066267 val_accuracy=0.786200 learning_rate=0.0100
Epoch:014 val

In [3]:
test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
print('Test dataset accuracy: %f' % test_acc)

Test dataset accuracy: 0.877300


### 模型读取恢复

In [4]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

ckpt_dir = './mnist_Tensorboard_tutorial_ckpt'
ckpt = tf.train.get_checkpoint_state(ckpt_dir)

if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print('Restore model from ' + ckpt.model_checkpoint_path)

INFO:tensorflow:Restoring parameters from ./mnist_Tensorboard_tutorial_ckpt\mnist_model_100.ckpt
Restore model from ./mnist_Tensorboard_tutorial_ckpt\mnist_model_100.ckpt


In [5]:
test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
print('Test dataset accuracy: %f' % test_acc)

Test dataset accuracy: 0.877300
